In [7]:
from nltk.corpus import wordnet as wn
import pandas as pd
from agreement_table import make_df

food, solid food
Counter({'baked_goods.n.01': 0.6692241988864567})
structure, construction
Counter({'bridge.n.01': 0.7547688869609703, 'hotel.n.01': 0.6833382475797337, 'house.n.01': 0.6651382074725258, 'housing.n.01': 0.6417861262151919, 'shelter.n.01': 0.6259479016251465, 'restaurant.n.01': 0.5102777997301323, 'outbuilding.n.01': 0.4967316375041171, 'place_of_worship.n.01': 0.46230158730158727})
tool
Counter({'tool.n.01': 0.8644481698403266})
vehicle
Counter({'train.n.01': 0.9264178075857339, 'boat.n.01': 0.7773438322261444, 'car.n.01': 0.7645554983259283, 'motorcycle.n.01': 0.7120078307771808, 'truck.n.01': 0.6901149216594108, 'ship.n.01': 0.6191029043196227, 'aircraft.n.01': 0.5237617222463469})
plant, flora, plant life
Counter({'flower.n.01': 0.7923828622358031})
food, nutrient
Counter({'edible_fruit.n.01': 0.7966810062674723, 'dish.n.02': 0.76834698791951, 'foodstuff.n.02': 0.6102027287190791, 'beverage.n.01': 0.5408680829932242, 'vegetable.n.01': 0.5190297118315694})
tableware
C

In [8]:
import os
import json
import sys
from collections import Counter
import numpy as np

In [98]:
fn = 'all_responses_round0-3_cleaned.csv'
resdf = make_df(fn)

In [99]:
resdf.head()

,Unnamed: 0,vg_img_id,cat,synset,vg_obj_name,responses_r0,opt-outs,vg_domain,top_response_domain_r0,responses_domains_r0,...,top_response_domain_r2,responses_domains_r3,responses_r3,top_response_domain_r3,vg_object_id,all_responses,clean,canon,spellchecked,vg_is_max
0,0,2357077,person,athlete.n.01,batter,"{'man': 4, 'batter': 2, 'baseball player': 1, ...",{},people,people,"{'people': 0.889, '$baseball player': 0.111}",...,people,"{'people': 0.75, '$bat': 0.125, 'clothing': 0....","{'player': 3, 'man': 2, 'batter': 1, 'bat': 1,...",people,815223,"Counter({'man': 11, 'batter': 8, 'player': 5, ...","{'man': 11, 'batter': 8, 'player': 5, 'person'...","{'man': 11, 'batter': 8, 'baseball player': 2,...","{'man': 11, 'batter': 8, 'baseball player': 2,...",0
1,1,2378759,article of clothing,overgarment.n.01,jacket,"{'jacket': 3, 'sweatshirt': 2, 'man': 2, 'hood...",{},clothing,clothing,"{'clothing': 0.333, '$sweatshirt': 0.222, 'peo...",...,people,"{'clothing': 0.556, 'people': 0.222, '$sweatsh...","{'jacket': 4, 'man': 2, 'sweatshirt': 1, 'hood...",clothing,1365136,"Counter({'jacket': 14, 'man': 11, 'sweatshirt'...","{'jacket': 14, 'man': 11, 'sweatshirt': 4, 'ho...","{'jacket': 14, 'sweatshirt': 4, 'man': 11, 'ho...","{'jacket': 14, 'sweatshirt': 4, 'man': 11, 'ho...",1
2,2,2319953,animal,ungulate.n.01,giraffe,{'giraffe': 7},{'#bbox': 2},animals_plants,animals_plants,{'animals_plants': 1.0},...,animals_plants,"{'animals_plants': 0.889, 'people': 0.111}","{'giraffe': 8, 'woman': 1}",animals_plants,3089834,"Counter({'giraffe': 32, 'woman': 1})","{'giraffe': 32, 'woman': 1}","{'giraffe': 32, 'woman': 1}","{'giraffe': 32, 'woman': 1}",1
3,3,2361401,animal,ungulate.n.01,giraffe,{'giraffe': 9},{},animals_plants,animals_plants,{'animals_plants': 1.0},...,animals_plants,{'animals_plants': 1.0},{'giraffe': 9},animals_plants,2363103,Counter({'giraffe': 36}),{'giraffe': 36},{'giraffe': 36},{'giraffe': 36},1
4,4,2401994,"food, nutrient",dish.n.02,sandwich,"{'sandwich': 6, 'food': 2, 'hotdog': 1}",{},food,food,"{'food': 0.889, '$hotdog': 0.111}",...,food,"{'food': 0.778, '$foot': 0.111, '$burger': 0.111}","{'food': 3, 'sandwich': 3, 'foot': 1, 'cake': ...",food,1141828,"Counter({'sandwich': 25, 'food': 6, 'hotdog': ...","{'sandwich': 25, 'food': 6, 'hotdog': 1, 'sand...","{'sandwich': 26, 'food': 7, 'hotdog': 1, 'cake...","{'sandwich': 26, 'food': 7, 'hotdog': 1, 'cake...",1


In [100]:
def name_pairs(rdict):
   
    np_count = Counter()
    if len(rdict.keys()) > 1:
        topname = rdict.most_common(1)[0][0]
        for oname in rdict:
            if not oname == topname:
                np_count[(topname,oname)] = rdict[oname]
                
    return np_count

In [101]:
paircount = Counter()
for ndict in resdf['spellchecked']:
    #print(ndict)
    x = name_pairs(ndict)
    paircount.update(x)

In [102]:
paircount.most_common(20)

[(('airplane', 'plane'), 11194),
 (('couch', 'sofa'), 4090),
 (('plane', 'airplane'), 3829),
 (('woman', 'person'), 3611),
 (('man', 'person'), 3558),
 (('desk', 'table'), 3448),
 (('boy', 'child'), 3249),
 (('shirt', 't-shirt'), 2928),
 (('motorcycle', 'bike'), 2632),
 (('jacket', 'coat'), 2402),
 (('woman', 'girl'), 2338),
 (('girl', 'child'), 1987),
 (('pizza', 'food'), 1890),
 (('carpet', 'floor'), 1697),
 (('jacket', 'shirt'), 1552),
 (('bench', 'chair'), 1401),
 (('desk', 'keyboard'), 1380),
 (('airplane', 'jet'), 1319),
 (('boat', 'ship'), 1301),
 (('woman', 'tennis player'), 1277)]

In [103]:
for ((top,other),freq) in paircount.most_common(20):
    topsyn = wn.synsets(top)
    othersyn = wn.synsets(other)
    print(top,topsyn)
    print(other,othersyn)
    print("***")

airplane [Synset('airplane.n.01')]
plane [Synset('airplane.n.01'), Synset('plane.n.02'), Synset('plane.n.03'), Synset('plane.n.04'), Synset('plane.n.05'), Synset('plane.v.01'), Synset('plane.v.02'), Synset('plane.v.03'), Synset('flat.s.01')]
***
couch [Synset('sofa.n.01'), Synset('couch.n.02'), Synset('couch.n.03'), Synset('frame.v.04')]
sofa [Synset('sofa.n.01')]
***
plane [Synset('airplane.n.01'), Synset('plane.n.02'), Synset('plane.n.03'), Synset('plane.n.04'), Synset('plane.n.05'), Synset('plane.v.01'), Synset('plane.v.02'), Synset('plane.v.03'), Synset('flat.s.01')]
airplane [Synset('airplane.n.01')]
***
woman [Synset('woman.n.01'), Synset('woman.n.02'), Synset('charwoman.n.01'), Synset('womanhood.n.02')]
person [Synset('person.n.01'), Synset('person.n.02'), Synset('person.n.03')]
***
man [Synset('man.n.01'), Synset('serviceman.n.01'), Synset('man.n.03'), Synset('homo.n.02'), Synset('man.n.05'), Synset('man.n.06'), Synset('valet.n.01'), Synset('man.n.08'), Synset('man.n.09'), Syns

In [104]:
hyp = lambda s:s.hypernyms()
list(sp.closure(hyp,depth=4))

[Synset('causal_agent.n.01'),
 Synset('organism.n.01'),
 Synset('physical_entity.n.01'),
 Synset('living_thing.n.01'),
 Synset('entity.n.01'),
 Synset('whole.n.02'),
 Synset('object.n.01')]

In [128]:
hyp = lambda s:s.hypernyms()

def is_hyponym(w1,w2,max_depth=5):
    
    for dpt in range(1,max_depth):
        
        syns1 = wn.synsets(w1,pos="n")
        syns2 = wn.synsets(w2,pos="n")
        
        for syn1 in syns1:
            hyp_closure = list(syn1.closure(hyp,depth=dpt))    
            
            for syn2 in syns2:
                if syn2 in hyp_closure:
                    return dpt
    return 0

def is_hypernym(w1,w2,max_depth=5):
    
    for dpt in range(1,max_depth):
        
        syns1 = wn.synsets(w1,pos="n")
        syns2 = wn.synsets(w2,pos="n")
        
        for syn2 in syns2:
            hyp_closure = list(syn2.closure(hyp,depth=dpt))    
            
            for syn1 in syns1:
                if syn1 in hyp_closure:
                    return dpt
    return 0

def is_synonym(w1,w2):
        
    syns1 = wn.synsets(w1,pos="n")
    syns2 = wn.synsets(w2,pos="n")
        
    for syn2 in syns2:   
            
        for syn1 in syns1:
                if syn1 == syn2:
                    return 1
    return 0

def is_cohyponym(w1,w2,max_depth=5):
    
    for dpt in range(1,6):
        
        syns1 = wn.synsets(w1,pos="n")
        syns2 = wn.synsets(w2,pos="n")
        
        
        
        
        for syn2 in syns2:
            
            hyp_closure2 = list(syn2.closure(hyp,depth=dpt))
              
            
            for syn1 in syns1:
                hyp_closure1 = list(syn1.closure(hyp,depth=dpt))
                
                for h2 in hyp_closure2:
                    for h1 in hyp_closure1:
                        if h1 == h2:
                            return dpt
    return 0

In [106]:
is_hyponym("person","man")

0

In [107]:
is_hypernym("person","man")

1

In [129]:
pair_depth = {}

for (top,other) in paircount:
    if ' ' in top:
        top = '_'.join(top.split(' '))
    if ' ' in other:
        other = '_'.join(other.split(' '))
    if len(wn.synsets(top)) > 0 and \
        len(wn.synsets(other)) > 0:
        pair_depth[(top,other)] = (is_synonym(top,other),\
                               is_hyponym(top,other,max_depth=10),\
                               is_hypernym(top,other,max_depth=10),\
                               is_cohyponym(top,other,max_depth=10)
                              )
    else:
        print("WORD NOUT FOUND in WN")
        print(top,other,len(wn.synsets(top)),len(wn.synsets(other)))

WORD NOUT FOUND in WN
jacket hoodie 7 0
WORD NOUT FOUND in WN
van motorized_vehicle 5 0
WORD NOUT FOUND in WN
desk computer_desk 1 0
WORD NOUT FOUND in WN
truck trash_truck 3 0
WORD NOUT FOUND in WN
pizza pizza_toppings 1 0
WORD NOUT FOUND in WN
dugout dugout_fence 3 0
WORD NOUT FOUND in WN
dugout warning_track 3 0
WORD NOUT FOUND in WN
train sub_train 17 0
WORD NOUT FOUND in WN
truck rescue_truck 3 0
WORD NOUT FOUND in WN
plane army_plane 9 0
WORD NOUT FOUND in WN
plane fighter_jet 9 0
WORD NOUT FOUND in WN
house townhouse 14 0
WORD NOUT FOUND in WN
train subway_car 17 0
WORD NOUT FOUND in WN
skateboarder skate_board 1 0
WORD NOUT FOUND in WN
plane space_shuttle_carrier 9 0
WORD NOUT FOUND in WN
zebra carse 1 0
WORD NOUT FOUND in WN
jacket gamers 7 0
WORD NOUT FOUND in WN
woman sports_lady 4 0
WORD NOUT FOUND in WN
woman sports_dress 4 0
WORD NOUT FOUND in WN
floor bed_skirt 12 0
WORD NOUT FOUND in WN
pizza pizza_slice 1 0
WORD NOUT FOUND in WN
pizza pepperoni_pizza_slice 1 0
WORD NOU

WORD NOUT FOUND in WN
shirt zipangu 2 0
WORD NOUT FOUND in WN
food egg_breakfast 3 0
WORD NOUT FOUND in WN
train big_train 17 0
WORD NOUT FOUND in WN
man sports_man 13 0
WORD NOUT FOUND in WN
train train_engine 17 0
WORD NOUT FOUND in WN
desk computer_desktop 1 0
WORD NOUT FOUND in WN
couch track_pant 4 0
WORD NOUT FOUND in WN
bed upper_bed 13 0
WORD NOUT FOUND in WN
cow ae 4 0
WORD NOUT FOUND in WN
cow bullet_cart 4 0
WORD NOUT FOUND in WN
man wetsuit 13 0
WORD NOUT FOUND in WN
woman sports_man 4 0
WORD NOUT FOUND in WN
car antique_car 5 0
WORD NOUT FOUND in WN
car old_car 5 0
WORD NOUT FOUND in WN
man hoodie 13 0
WORD NOUT FOUND in WN
man skating_board 13 0
WORD NOUT FOUND in WN
bench sheltered_bench 9 0
WORD NOUT FOUND in WN
bench resting_place 9 0
WORD NOUT FOUND in WN
shirt red_shirt 2 0
WORD NOUT FOUND in WN
apple green_apple 2 0
WORD NOUT FOUND in WN
jacket motor_sport 7 0
WORD NOUT FOUND in WN
pan stove_top 7 0
WORD NOUT FOUND in WN
bird block_sparrow 6 0
WORD NOUT FOUND in WN


WORD NOUT FOUND in WN
skier bi_athlete 1 0
WORD NOUT FOUND in WN
suit man's_tie 10 0
WORD NOUT FOUND in WN
desk key_board 1 0
WORD NOUT FOUND in WN
desk lap_tops 1 0
WORD NOUT FOUND in WN
china fine_china 4 0
WORD NOUT FOUND in WN
china serving_ware 4 0
WORD NOUT FOUND in WN
china serving_set 4 0
WORD NOUT FOUND in WN
china expensive_china 4 0
WORD NOUT FOUND in WN
grill grill_bar 4 0
WORD NOUT FOUND in WN
table plate_of_food 8 0
WORD NOUT FOUND in WN
bridge electrical_line 12 0
WORD NOUT FOUND in WN
umpire umpire_baseball 3 0
WORD NOUT FOUND in WN
woman eye_class 4 0
WORD NOUT FOUND in WN
table stuffed_toy 8 0
WORD NOUT FOUND in WN
woman groomer 4 0
WORD NOUT FOUND in WN
tray place_set 1 0
WORD NOUT FOUND in WN
tray serving_tray 1 0
WORD NOUT FOUND in WN
counter kitchen_bar 13 0
WORD NOUT FOUND in WN
shirt shirt_and_tie 2 0
WORD NOUT FOUND in WN
woman eyeleteer 4 0
WORD NOUT FOUND in WN
plane air_plan 9 0
WORD NOUT FOUND in WN
train fuel_tank 17 0
WORD NOUT FOUND in WN
train train_tan

WORD NOUT FOUND in WN
skier hand_stick 1 0
WORD NOUT FOUND in WN
bed pet_bed 13 0
WORD NOUT FOUND in WN
bed cat_bed 13 0
WORD NOUT FOUND in WN
bed dog_bed 13 0
WORD NOUT FOUND in WN
man hot_bun 13 0
WORD NOUT FOUND in WN
dresser travel_bag 5 0
WORD NOUT FOUND in WN
dresser entertainment_system 5 0
WORD NOUT FOUND in WN
dresser closet_drawer 5 0
WORD NOUT FOUND in WN
dresser ward_row 5 0
WORD NOUT FOUND in WN
train wind_shield 17 0
WORD NOUT FOUND in WN
food long_john 3 0
WORD NOUT FOUND in WN
car smart_car 5 0
WORD NOUT FOUND in WN
man baseman 13 0
WORD NOUT FOUND in WN
bathtub bath_tub 1 0
WORD NOUT FOUND in WN
woman selfie 4 0
WORD NOUT FOUND in WN
table left_overs 8 0
WORD NOUT FOUND in WN
man long_sleeve_shirt 13 0
WORD NOUT FOUND in WN
truck number_plate 3 0
WORD NOUT FOUND in WN
pizza pizza_box 1 0
WORD NOUT FOUND in WN
baseball_player sports_man 1 0
WORD NOUT FOUND in WN
counter kitchen_worktop 13 0
WORD NOUT FOUND in WN
shirt slotter 2 0
WORD NOUT FOUND in WN
train taine 17 0
W

WORD NOUT FOUND in WN
dress tennis_clothes 21 0
WORD NOUT FOUND in WN
dress tennis_attire 21 0
WORD NOUT FOUND in WN
bowl basket_of_fruit 12 0
WORD NOUT FOUND in WN
vest suit_vest 7 0
WORD NOUT FOUND in WN
dock wooden_bridge 12 0
WORD NOUT FOUND in WN
tracks train_tracks 16 0
WORD NOUT FOUND in WN
cow black_cow 4 0
WORD NOUT FOUND in WN
giraffe geffen 1 0
WORD NOUT FOUND in WN
woman mother_and_son 4 0
WORD NOUT FOUND in WN
fruit serving_fruit_tray 5 0
WORD NOUT FOUND in WN
water polar_beer 10 0
WORD NOUT FOUND in WN
snowboarder sky_board 1 0
WORD NOUT FOUND in WN
snowboarder skate_board 1 0
WORD NOUT FOUND in WN
shirt night_gown 2 0
WORD NOUT FOUND in WN
woman cycler 4 0
WORD NOUT FOUND in WN
refrigerator refrigerated_aisle 1 0
WORD NOUT FOUND in WN
refrigerator bottle_showcase 1 0
WORD NOUT FOUND in WN
hat head_wrap 4 0
WORD NOUT FOUND in WN
shirt baby_doll_dress 2 0
WORD NOUT FOUND in WN
batter baseball_man 5 0
WORD NOUT FOUND in WN
table area_rug 8 0
WORD NOUT FOUND in WN
bowl servi

...

WORD NOUT FOUND in WN
girl toy_equipment 5 0
WORD NOUT FOUND in WN
bed bed,_pillow,_dog 13 0
WORD NOUT FOUND in WN
woman pants,_belt,_shirt 4 0
WORD NOUT FOUND in WN
woman ord 4 0
WORD NOUT FOUND in WN
table cheese_ban 8 0
WORD NOUT FOUND in WN
vase flower_doll 1 0
WORD NOUT FOUND in WN
toy stuffed_lamb 8 0
WORD NOUT FOUND in WN
toy stuffed_animal_toy 8 0
WORD NOUT FOUND in WN
woman gjr, 4 0
WORD NOUT FOUND in WN
cup food_cup 11 0
WORD NOUT FOUND in WN
cup yogurt_cup 11 0
WORD NOUT FOUND in WN
bench ambary 9 0
WORD NOUT FOUND in WN
horse rider_and_horse 6 0
WORD NOUT FOUND in WN
dog phthalocyanine 8 0
WORD NOUT FOUND in WN
man sandwich,_glasses,_shirt,_wristwatch 13 0
WORD NOUT FOUND in WN
bird crane_bird 6 0
WORD NOUT FOUND in WN
motorcycle handle_bar 2 0
WORD NOUT FOUND in WN
motorcycle bike_glass 2 0
WORD NOUT FOUND in WN
chair catechizer 7 0
WORD NOUT FOUND in WN
baseball_player baseball_hat 1 0
WORD NOUT FOUND in WN
hotdog hotdogged 3 0
WORD NOUT FOUND in WN
boy dress_sh 4 0
WORD 

WORD NOUT FOUND in WN
boy surfer_boy 4 0
WORD NOUT FOUND in WN
jacket riding_jacket 7 0
WORD NOUT FOUND in WN
cabinet modern_kitchen 4 0
WORD NOUT FOUND in WN
cake old_birthday 4 0
WORD NOUT FOUND in WN
cake cake_and_candles 4 0
WORD NOUT FOUND in WN
train terai 17 0
WORD NOUT FOUND in WN
tent vendor_tent 3 0
WORD NOUT FOUND in WN
tent tent_covering 3 0
WORD NOUT FOUND in WN
bed scan_machine 13 0
WORD NOUT FOUND in WN
girl flayer 5 0
WORD NOUT FOUND in WN
floor floor_mat 12 0
WORD NOUT FOUND in WN
boat wind_mast 3 0
WORD NOUT FOUND in WN
marina docking_area 1 0
WORD NOUT FOUND in WN
bird yellow_bird 6 0
WORD NOUT FOUND in WN
hotdog paper_cloth 3 0
WORD NOUT FOUND in WN
hotdog sandwich_roll 3 0
WORD NOUT FOUND in WN
jacket sun_glass 7 0
WORD NOUT FOUND in WN
bridge rail_road 12 0
WORD NOUT FOUND in WN
donut wadai 1 0
WORD NOUT FOUND in WN
pizza mushroom_topping 1 0
WORD NOUT FOUND in WN
man flayer 13 0
WORD NOUT FOUND in WN
bowl cast_iron_pan 12 0
WORD NOUT FOUND in WN
birdbath bird_wat

WORD NOUT FOUND in WN
buffet buffet_case 5 0
WORD NOUT FOUND in WN
buffet serving_station 5 0
WORD NOUT FOUND in WN
buffet food_hot_bar 5 0
WORD NOUT FOUND in WN
counter table_cupboard 13 0
WORD NOUT FOUND in WN
woman coffee_pot 4 0
WORD NOUT FOUND in WN
bucket bu_gate 4 0
WORD NOUT FOUND in WN
floor hatstand 12 0
WORD NOUT FOUND in WN
shirt anime_shirt 2 0
WORD NOUT FOUND in WN
table vase_with_flowers 8 0
WORD NOUT FOUND in WN
vase bot_til 1 0
WORD NOUT FOUND in WN
sandwich cake_slice 3 0
WORD NOUT FOUND in WN
zebra wooden_table 1 0
WORD NOUT FOUND in WN
bricks brick_wall 2 0
WORD NOUT FOUND in WN
desk computer_equipment 1 0
WORD NOUT FOUND in WN
man shier_board 13 0
WORD NOUT FOUND in WN
carpet under_desk_area 5 0
WORD NOUT FOUND in WN
shirt purse_strap 2 0
WORD NOUT FOUND in WN
person dree 3 0
WORD NOUT FOUND in WN
horse man_on_horseback 6 0
WORD NOUT FOUND in WN
building historic_hotel 14 0
WORD NOUT FOUND in WN
jacket pec 7 0
WORD NOUT FOUND in WN
oatmeal rice_pudding 2 0
WORD NOU

WORD NOUT FOUND in WN
woman wine_glasses 4 0
WORD NOUT FOUND in WN
jet 757 9 0
WORD NOUT FOUND in WN
truck storage_vehicle 3 0
WORD NOUT FOUND in WN
woman ambary 4 0
WORD NOUT FOUND in WN
oven michele 1 0
WORD NOUT FOUND in WN
oven food_maker 1 0
WORD NOUT FOUND in WN
oven oven_door 1 0
WORD NOUT FOUND in WN
shirt black_tea_shirt 2 0
WORD NOUT FOUND in WN
counter scat_board 13 0
WORD NOUT FOUND in WN
counter display_stand 13 0
WORD NOUT FOUND in WN
counter vehicle_accessories 13 0
WORD NOUT FOUND in WN
counter show_things 13 0
WORD NOUT FOUND in WN
man skate_bard 13 0
WORD NOUT FOUND in WN
bench news_paper 9 0
WORD NOUT FOUND in WN
desk cool_drinks 1 0
WORD NOUT FOUND in WN
food fruit_roll 3 0
WORD NOUT FOUND in WN
garage store_room 3 0
WORD NOUT FOUND in WN
cake smiley_face 4 0
WORD NOUT FOUND in WN
boy slotter 4 0
WORD NOUT FOUND in WN
bowl bowl_of_cereal 12 0
WORD NOUT FOUND in WN
shirt shirt_rain 2 0
WORD NOUT FOUND in WN
shirt t_shat 2 0
WORD NOUT FOUND in WN
bowl avocado_salad 12

WORD NOUT FOUND in WN
jacket husband_and_wife 7 0
WORD NOUT FOUND in WN
counter meal_prep 13 0
WORD NOUT FOUND in WN
truck frisbee_player 3 0
WORD NOUT FOUND in WN
hat small_cupcakes 4 0
WORD NOUT FOUND in WN
couch x_mas_tree 4 0
WORD NOUT FOUND in WN
train loup 17 0
WORD NOUT FOUND in WN
bed bi_edged 13 0
WORD NOUT FOUND in WN
shirt button_down 2 0
WORD NOUT FOUND in WN
shirt game_controller 2 0
WORD NOUT FOUND in WN
bride ear_rings 3 0
WORD NOUT FOUND in WN
truck satellite_truck 3 0
WORD NOUT FOUND in WN
shirt athletic_shirt 2 0
WORD NOUT FOUND in WN
building rain_gutter 14 0
WORD NOUT FOUND in WN
building water_spout 14 0
WORD NOUT FOUND in WN
bridge roup 12 0
WORD NOUT FOUND in WN
bridge tire_an 12 0
WORD NOUT FOUND in WN
train tire_ann 17 0
WORD NOUT FOUND in WN
bathtub bathtub_side 1 0
WORD NOUT FOUND in WN
desk table_things 1 0
WORD NOUT FOUND in WN
tub treen 3 0
WORD NOUT FOUND in WN
desk desk_stand 1 0
WORD NOUT FOUND in WN
truck trophy_truck 3 0
WORD NOUT FOUND in WN
truck mo

In [130]:
pair_depth

{('man', 'batter'): (0, 0, 0, 4),
 ('man', 'baseball_player'): (0, 0, 0, 3),
 ('man', 'player'): (0, 0, 0, 1),
 ('man', 'person'): (0, 1, 0, 2),
 ('man', 'helmet'): (0, 0, 0, 4),
 ('man', 'boy'): (0, 0, 1, 1),
 ('man', 'uniform'): (0, 0, 0, 4),
 ('man', 'bat'): (0, 0, 0, 3),
 ('jacket', 'sweatshirt'): (0, 0, 0, 3),
 ('jacket', 'man'): (0, 0, 0, 3),
 ('jacket', 'person'): (0, 0, 0, 4),
 ('jacket', 'shirt'): (0, 0, 0, 3),
 ('giraffe', 'woman'): (0, 0, 0, 0),
 ('sandwich', 'food'): (0, 4, 0, 5),
 ('sandwich', 'hotdog'): (0, 0, 1, 2),
 ('sandwich', 'cake'): (0, 0, 0, 2),
 ('sandwich', 'burger'): (0, 0, 1, 2),
 ('van', 'automobile'): (0, 0, 0, 2),
 ('van', 'car'): (0, 1, 0, 2),
 ('van', 'tailgate'): (0, 0, 0, 0),
 ('van', 'truck'): (0, 1, 0, 2),
 ('van', 'bus'): (0, 0, 0, 2),
 ('van', 'vehicle'): (0, 3, 0, 4),
 ('van', 'suitcase'): (0, 0, 0, 3),
 ('car', 'automobile'): (1, 0, 0, 1),
 ('car', 'vehicle'): (0, 2, 0, 3),
 ('car', 'van'): (0, 0, 1, 2),
 ('desk', 'curtain'): (0, 0, 0, 3),
 ('desk

In [226]:
pair_rel = {}
no_rel = Counter()
have_rel = Counter()
for p in paircount:
    if p in pair_depth:
        if pair_depth[p] == (0,0,0,0):
            print(p)
            no_rel[p] = paircount[p]
        else:
            pair_rel[p] = pair_depth[p]
            have_rel[p] = paircount[p]
    else:
        no_rel[p] = paircount[p]

('giraffe', 'woman')
('van', 'tailgate')
('desk', 'electronics')
('woman', 'mom')
('woman', 'oven')
('pizza', 'toppings')
('salad', 'bean')
('salad', 'carrot')
('dugout', 'dirt')
('shirt', 'controllership')
('girl', 't-shirt')
('woman', 't-shirt')
('floor', 'dining')
('donut', 'glaze')
('jacket', 'people')
('table', 'sunflower')
('boy', 'sunglasses')
('bench', 'rhombencephalon')
('boy', 't-shirt')
('child', 'umbrella')
('lettuce', 'salad')
('girl', 'human')
('sandwich', 'tomato')
('duck', 'swam')
('dress', 'torso')
('salad', 'broccoli')
('hood', 'automobile')
('knife', 'food')
('skater', 'skateboard')
('couch', 'player')
('shirt', 'polo')
('girl', 'donut')
('girl', 'eat')
('skier', 'skateboard')
('skier', 'helmet')
('skier', 'icebreaker')
('dress', 'tennis')
('girl', 'burger')
('child', 'portrait')
('giraffe', 'neck')
('crust', 'spatula')
('crust', 'pizza')
('crust', 'platter')
('man', 'scatter')
('man', 'frizzle')
('counter', 'shelve')
('building', 'clock')
('building', 'restart')
('t

('table', 'welding')
('motorcycle', 'fairing')
('skateboarder', 'top')
('boat', 'sea')
('car', 'corolla')
('snow', 'pant')
('snow', 't-shirt')
('snow', 'ski')
('snow', 'skate')
('remote', 'recliner')
('cabinets', 'oven')
('boat', 'food')
('bathtub', 'washermen')
('bed', 'pillowcase')
('airplane', 'shoe')
('airplane', 'pilot')
('dog', 'airplane')
('car', 'tree')
('cabinet', 'ta')
('surfer', 'shirty')
('soda', 'bottle')
('vase', 'airplane')
('woman', 'see')
('desk', 'switcher')
('desk', 'soundboard')
('bed', 'tell')
('toilet', 'lindane')
('toilet', 'sing')
('toilet', 'tala')
('cup', 'upholder')
('car', 'machination')
('table', 'mealtime')
('bridge', 'see')
('device', 'electronic')
('device', 'unsporting')
('woman', 'careerism')
('cat', 'unclear')
('carpet', 'controllership')
('kitchen', 'shelving')
('grass', 'shoe')
('grass', 'top')
('grass', 'floor')
('grass', 'yard')
('grass', 'field')
('cow', 'wheel')
('house', 'ship')
('grass', 'pant')
('grass', 't-shirt')
('grass', 'shirt')
('grass'

In [168]:
len(no_rel),len(have_rel)

(10349, 13376)

In [169]:
sum(no_rel.values())/sum(paircount.values())

0.16004039291287983

In [170]:
sum(have_rel.values())

228742

In [193]:
unordered_counts = Counter()
unordered_rel = {}
unordered_depth = {}
swapped = []
for (p1,p2) in have_rel:
    swapp= (p2,p1)
    if not (p1,p2) in swapped:
        unordered_counts[(p1,p2)] = have_rel[(p1,p2)]
        unordered_counts[(p1,p2)] += have_rel[swapp]
        if pair_rel[(p1,p2)][0] > 0:
            unordered_rel[(p1,p2)] = 'synonyms'
            unordered_depth[(p1,p2)] = pair_rel[(p1,p2)][0]
        elif pair_rel[(p1,p2)][1] > 0:
            unordered_rel[(p1,p2)] = 'hierarchical'
            unordered_depth[(p1,p2)] = pair_rel[(p1,p2)][1]
        elif pair_rel[(p1,p2)][2] > 0:
            unordered_rel[(p1,p2)] = 'hierarchical'
            unordered_depth[(p1,p2)] = pair_rel[(p1,p2)][2]
        elif pair_rel[(p1,p2)][3] > 0:
            unordered_rel[(p1,p2)] = 'crossclassified'
            unordered_depth[(p1,p2)] = pair_rel[(p1,p2)][3]
            
        swapped.append(swapp)

In [179]:
sum(unordered_counts.values())

228742

In [180]:
sum(have_rel.values())

228742

In [181]:
len(unordered_rel)

12093

In [183]:
len(have_rel)

13376

In [196]:
typecounts = Counter()
tokencounts = Counter()
depth_av = {}
for p in unordered_rel:
    if unordered_rel[p] not in depth_av:
        depth_av[unordered_rel[p]] = []
    typecounts[unordered_rel[p]] += 1
    tokencounts[unordered_rel[p]] += unordered_counts[p]
    depth_av[unordered_rel[p]].append(unordered_depth[p])

In [197]:
typecounts

Counter({'crossclassified': 10748, 'hierarchical': 1167, 'synonyms': 178})

In [198]:
tokencounts

Counter({'crossclassified': 125930, 'hierarchical': 75043, 'synonyms': 27769})

In [220]:
outdf = []
totaltypes = sum(typecounts.values())
totaltokens = sum(tokencounts.values())
for p in typecounts:
    outdf.append((p,"%.3f"%(typecounts[p]/totaltypes),"%.3f"%(tokencounts[p]/totaltokens),"%.3f"%(np.mean(depth_av[p]))))

In [221]:
outdff = pd.DataFrame(outdf,columns=['relation','types','tokens','av WN depth'])

In [222]:
outdff

,relation,types,tokens,av WN depth
0,crossclassified,0.889,0.551,3.479
1,hierarchical,0.097,0.328,2.204
2,synonyms,0.015,0.121,1.000


In [225]:
print(outdff.to_latex(index=False))

\begin{tabular}{llll}
\toprule
        relation &  types & tokens & av WN depth \\
\midrule
 crossclassified &  0.889 &  0.551 &       3.479 \\
    hierarchical &  0.097 &  0.328 &       2.204 \\
        synonyms &  0.015 &  0.121 &       1.000 \\
\bottomrule
\end{tabular}



In [231]:
cat2paircount = {}
for cat in set(list(resdf['cat'])):
    cat2paircount[cat] = Counter()
    catdf = resdf[resdf['cat'] == cat]
    
    for ndict in catdf['spellchecked']:
    #print(ndict)
        x = name_pairs(ndict)
        cat2paircount[cat].update(x)

In [243]:
examples = []
for cat in cat2paircount:
    exstr = []
    print(cat)
    for ((n1,n2),freq) in cat2paircount[cat].most_common(10):
        exstr.append("%s -- %s (%d)"%(n1,n2,freq))
    examples.append((cat,", ".join(exstr)))
    

food, solid food
structure, construction
tool
vehicle
plant, flora, plant life
food, nutrient
tableware
article of clothing
animal
ware
person
instrumentality, instrumentation


In [244]:
exdf = pd.DataFrame(examples,columns=['category',"most frequent naming variants"])

In [245]:
exdf

,category,most frequent naming variants
0,"food, solid food","sandwich -- food (492), hotdog -- food (467), hotdog -- sandwich (318), cake -- food (265), bread -- sandwich (257), bread -- food (247), bread -- bun (206), donut -- food (188), hotdog -- bun (184), sandwich -- burger (163)"
1,"structure, construction","house -- building (1160), building -- house (511), bridge -- train (326), bridge -- overpass (235), house -- window (161), house -- home (123), tent -- canopy (120), building -- castle (101), bridge -- building (98), bridge -- pole (85)"
2,tool,"knife -- pizza (27), knife -- food (22), knife -- cake (17), knife -- banana (16), knife -- plate (15), knife -- apple (13), knife -- pocket knife (12), knife -- butter knife (12), knife -- table (10), knife -- carrot (8)"
3,vehicle,"airplane -- plane (11194), plane -- airplane (3829), motorcycle -- bike (2624), airplane -- jet (1319), boat -- ship (1301), truck -- car (1095), car -- vehicle (874), motorcycle -- wheel (861), truck -- vehicle (718), truck -- wheel (716)"
4,"plant, flora, plant life","flower -- flowers (29), rose -- flower (25), flower -- rose (22), flower -- vase (18), flowers -- flower (14), flower -- dandelion (14), plant -- flower (12), flower -- sunflower (11), plant -- wheat (11), flower -- flower pot (10)"
5,"food, nutrient","pizza -- food (1796), sandwich -- food (631), pizza -- cheese (447), pizza -- plate (423), salad -- food (402), sandwich -- burger (235), pizza -- toppings (217), pizza -- pizza slice (162), sandwich -- bread (156), food -- sandwich (147)"
6,tableware,"cup -- mug (146), mug -- cup (104), cup -- coffee (93), cup -- drink (73), bowl -- food (60), cup -- glass (39), bowl -- cup (36), glass -- wine glass (35), glass -- cup (34), cup -- coffee cup (31)"
7,article of clothing,"shirt -- t-shirt (2914), jacket -- coat (2396), jacket -- shirt (1552), jacket -- suit (1168), suit -- jacket (1029), shirt -- jacket (813), shirt -- tie (723), shirt -- man (487), shirt -- dress (462), shirt -- sweater (450)"
8,animal,"cow -- bull (515), sheep -- goat (486), cow -- animal (445), giraffe -- animal (380), bird -- parrot (349), sheep -- animal (294), sheep -- lamb (282), horse -- animal (269), cat -- animal (237), bird -- seagull (231)"
9,ware,"fork -- spoon (71), fork -- plate (44), spoon -- food (32), fork -- food (31), fork -- cake (21), spoon -- fork (18), spoon -- wooden spoon (15), fork -- broccoli (15), fork -- silverware (10), spoon -- vegetables (8)"


In [246]:
pd.set_option('display.max_colwidth', -1)
print(exdf.to_latex(index=False))

\begin{tabular}{ll}
\toprule
                         category &                                                                                                                                                                                                                    most frequent naming variants \\
\midrule
 food, solid food &  sandwich -- food (492), hotdog -- food (467), hotdog -- sandwich (318), cake -- food (265), bread -- sandwich (257), bread -- food (247), bread -- bun (206), donut -- food (188), hotdog -- bun (184), sandwich -- burger (163) \\
 structure, construction &  house -- building (1160), building -- house (511), bridge -- train (326), bridge -- overpass (235), house -- window (161), house -- home (123), tent -- canopy (120), building -- castle (101), bridge -- building (98), bridge -- pole (85) \\
 tool &  knife -- pizza (27), knife -- food (22), knife -- cake (17), knife -- banana (16), knife -- plate (15), knife -- apple (13), knife -- pocket knife (12), kni

In [252]:
for (n1,n2) in unordered_rel:
    if unordered_rel[(n1,n2)] == "hierarchical":
        print(n1,n2, paircount[(n1,n2)],paircount[(n2,n1)])

man person 3558 205
man boy 1215 467
sandwich food 1123 262
sandwich hotdog 175 365
sandwich burger 398 90
van car 299 242
van truck 183 439
van vehicle 155 0
car vehicle 874 7
child boy 1015 3249
child person 220 19
woman girl 2338 405
woman lady 798 0
jacket coat 2402 352
jacket clothing 235 0
jacket parka 39 0
pizza food 1890 114
truck vehicle 718 2
curtain blind 7 0
man guy 385 0
man male 228 0
salad food 412 73
motorcycle motorbike 447 0
motorcycle vehicle 222 0
bird crow 62 0
bird owl 29 33
bird eagle 73 22
boy person 800 67
boy kid 1096 0
woman person 3611 177
airplane jet 1319 172
airplane aircraft 309 0
shirt t-shirt 2928 1
plane jet 349 154
cake food 290 82
girl child 1987 646
girl kid 385 0
girl person 387 37
robe bathrobe 29 0
robe clothing 11 0
skateboarder skater 1194 554
skateboarder person 538 19
cat kitty 23 0
cat animal 239 2
house building 1160 516
plane aircraft 118 0
cow animal 446 9
cow oxen 52 0
woman female 261 0
shirt clothing 324 0
couch furniture 158 8
donut 

dish container 4 0
animal reindeer 4 0
animal moose 1 3
animal deer 4 0
dog chow 1 0
food burrito 14 3
peacock animal 1 0
dog terrier 2 0
fruit oranges 10 0
fruit apricot 2 0
fruit peaches 1 0
fruit peach 1 0
cow mammal 4 0
cake biscuit 2 0
canopy cloth 1 0
canopy awning 3 3
sandwich grinder 1 0
jacket garment 3 0
food fries 3 0
boat pontoon 4 0
ship vessel 1 0
ship battleship 1 0
bagel bun 13 0
fork cutlery 3 0
food bite 2 0
food granola 5 0
food meals 1 0
pitcher container 1 0
table foods 5 0
lettuce produce 1 0
trough container 1 0
van camper 2 0
car vehicles 7 0
shirt turtleneck 9 0
cabinet furnishing 1 0
glass glassware 1 0
player hitter 3 0
pot kettle 3 4
pot cauldron 1 0
pot cookware 2 0
road highway 2 0
woman maid 3 0
bed wall 5 5
blanket bedding 1 0
goose waterbird 1 0
salad slaw 2 0
salad coleslaw 7 9
woman baby 10 0
bear animal 16 10
wine liquid 1 0
building palace 20 0
boat vehicles 5 0
bird toucan 26 0
house chalet 2 0
field land 1 0
field steppe 1 0
woman ladies 2 0
house

In [249]:
unordered_rel

{('man', 'batter'): 'crossclassified',
 ('man', 'player'): 'crossclassified',
 ('man', 'person'): 'hierarchical',
 ('man', 'helmet'): 'crossclassified',
 ('man', 'boy'): 'hierarchical',
 ('man', 'uniform'): 'crossclassified',
 ('man', 'bat'): 'crossclassified',
 ('jacket', 'sweatshirt'): 'crossclassified',
 ('jacket', 'man'): 'crossclassified',
 ('jacket', 'person'): 'crossclassified',
 ('jacket', 'shirt'): 'crossclassified',
 ('sandwich', 'food'): 'hierarchical',
 ('sandwich', 'hotdog'): 'hierarchical',
 ('sandwich', 'cake'): 'crossclassified',
 ('sandwich', 'burger'): 'hierarchical',
 ('van', 'automobile'): 'crossclassified',
 ('van', 'car'): 'hierarchical',
 ('van', 'truck'): 'hierarchical',
 ('van', 'bus'): 'crossclassified',
 ('van', 'vehicle'): 'hierarchical',
 ('van', 'suitcase'): 'crossclassified',
 ('car', 'automobile'): 'synonyms',
 ('car', 'vehicle'): 'hierarchical',
 ('desk', 'curtain'): 'crossclassified',
 ('desk', 'keyboard'): 'crossclassified',
 ('zebra', 'horse'): 'cros

In [132]:
paircount[('skier', 'helmet')]

46

In [134]:
wn.synsets('lettuce')

[Synset('boodle.n.01'), Synset('lettuce.n.02'), Synset('lettuce.n.03')]

In [136]:
is_hyponym('lettuce','salad',max_depth=20)

0

In [138]:
is_cohyponym('lettuce','salad',max_depth=10)

0

In [143]:
wn.synsets('fruit')

[Synset('fruit.n.01'),
 Synset('yield.n.03'),
 Synset('fruit.n.03'),
 Synset('fruit.v.01'),
 Synset('fruit.v.02')]

In [147]:
lettsyn = wn.synsets('food')
print(lettsyn)
hyp_closure2 = list(lettsyn[0].closure(hyp,depth=5))
print(hyp_closure2)

[Synset('food.n.01'), Synset('food.n.02'), Synset('food.n.03')]
[Synset('substance.n.07'), Synset('matter.n.03'), Synset('physical_entity.n.01'), Synset('entity.n.01')]


In [119]:
len(paircount)

23725

In [120]:
len(pair_depth)

18770

In [110]:
paircount[('hotdog','hot dog')]

0

In [111]:
wn.synsets('hotdog')

[Synset('hotdog.n.01'), Synset('hotdog.n.02'), Synset('frank.n.02')]

In [112]:
is_hyponym('giraffe','animal',max_depth=10)

8

In [113]:
wn.synsets('giraffe')

[Synset('giraffe.n.01')]

In [114]:
is_hyponym('flamingo','animal',max_depth=10)

6

In [126]:
wn.synsets('hat_stand')

[]

In [116]:
paircount[('skier','skiier')]

0